<a href="https://colab.research.google.com/github/shvmshri/ResNet/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os
from keras.initializers import glorot_uniform


import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [0]:


def identity_block_bottleneck(X, f, filters):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
  
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters= F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X =  Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X =  BatchNormalization(axis = 3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
def identity_block_normal(X,f,filters):
   
   # Retrieve Filters
    F1, F2 = filters

   # Save the input value
    X_shortcut = X

   # First component of main path 
    X = Conv2D(F1, (f, f), strides = (1,1),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

   # Second component of main path 
    X =  Conv2D(F2, (f, f), strides = (1,1),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)

   # Final step
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
def convolutional_block_bottleneck(X, f, filters, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s),padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X =  Conv2D(F2, (f, f), strides = (1,1),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    
    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
   
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X =  Add()([X,X_shortcut])
    X = Activation('relu')(X)

    
    return X

In [0]:
def convolutional_block_normal(X, f, filters, s = 2):
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path 
    X = Conv2D(F1, (f, f), strides = (1,1),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X=ZeroPadding2D((1,1))(X)
    X =  Conv2D(F2, (f, f), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    
    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(F2, (1, 1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
   
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X =  Add()([X,X_shortcut])
    X = Activation('relu')(X)

    
    return X

In [0]:
def ResNet_bottleneck(input_shape = (32, 32, 3), classes = 10):
  """
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input=Input(input_shape)
    X=X_input
    
    # Zero-Padding
            #  X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(16, (3, 3), strides = (2, 2),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    # X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    # X = Activation('relu')(X)
    

    # Stage 2
            # X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    for i in range(6):
       X = identity_block_bottleneck(X, 3, [8, 8, 16])
   
    # Stage 3
    X = convolutional_block_bottleneck(X, f=3,filters =[16,16,32])

    # Stage 4 
    for i in range(6):
       X = identity_block_bottleneck(X, 3, [16, 16, 32])

    # Stage 5 
    X = convolutional_block_bottleneck(X, f=3,filters =[32,32,64])

    #Stage 6
    for i in range(6):
       X = identity_block_bottleneck(X, 3, [32, 32, 64])
   
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size= (2,2),strides = (2,2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet_bottleneck')

    return model

In [0]:
model = ResNet_bottleneck()

In [0]:
 
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [67]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)


datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test,y_test), epochs=70)
scores= model.evaluate(x_test,y_test,verbose =1)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])



Epoch 1/70
391/391 [==============================] - 102s 260ms/step - loss: 2.2475 - accuracy: 0.1935 - val_loss: 1.9988 - val_accuracy: 0.2667
Epoch 2/70
391/391 [==============================] - 79s 203ms/step - loss: 1.9593 - accuracy: 0.2810 - val_loss: 1.8206 - val_accuracy: 0.3389
Epoch 3/70
391/391 [==============================] - 81s 207ms/step - loss: 1.8504 - accuracy: 0.3180 - val_loss: 1.7168 - val_accuracy: 0.3706
Epoch 4/70
391/391 [==============================] - 82s 210ms/step - loss: 1.7734 - accuracy: 0.3454 - val_loss: 1.6506 - val_accuracy: 0.3910
Epoch 5/70
391/391 [==============================] - 82s 210ms/step - loss: 1.7085 - accuracy: 0.3703 - val_loss: 1.5774 - val_accuracy: 0.4188
Epoch 6/70
391/391 [==============================] - 82s 210ms/step - loss: 1.6528 - accuracy: 0.3923 - val_loss: 1.5448 - val_accuracy: 0.4318
Epoch 7/70
391/391 [==============================] - 82s 210ms/step - loss: 1.6001 - accuracy: 0.4129 - val_loss: 1.5097 - val_a

model using normal blocks

In [0]:
def ResNet_normal(input_shape = (32, 32, 3), classes = 10):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    X=X_input
    X=MaxPooling2D(pool_size=(2,2),padding='same')(X)
    # Zero-Padding
            #  X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(16, (3, 3), strides = (2, 2),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
   # X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
   # X = Activation('relu')(X)
    

    # Stage 2
            # X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    for i in range(10):
       X = identity_block_normal(X, 3, [16,16])
   
    # Stage 3
    X = convolutional_block_normal(X, f=3,filters =[16,32])

    # Stage 4 
    for i in range(10):
       X = identity_block_normal(X, 3, [32, 32])

    # Stage 5 
    X = convolutional_block_normal(X, f=3,filters =[32,64])

    #Stage 6
    for i in range(10):
       X = identity_block_normal(X, 3, [64, 64])
   
    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size= (2,2),strides = (2,2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model2 = Model(inputs = X_input, outputs = X, name='ResNet_bottleneck')

    return model2

In [0]:
model2 = ResNet_normal()
model2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [91]:



model2.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test,y_test), epochs=70)
scores= model2.evaluate(x_test,y_test,verbose =1)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])


Epoch 1/70
391/391 [==============================] - 100s 255ms/step - loss: 2.0450 - accuracy: 0.2662 - val_loss: 1.7375 - val_accuracy: 0.3571
Epoch 2/70
391/391 [==============================] - 79s 203ms/step - loss: 1.7617 - accuracy: 0.3598 - val_loss: 1.6236 - val_accuracy: 0.4198
Epoch 3/70
391/391 [==============================] - 79s 203ms/step - loss: 1.6673 - accuracy: 0.3939 - val_loss: 1.5488 - val_accuracy: 0.4337
Epoch 4/70
391/391 [==============================] - 79s 203ms/step - loss: 1.6276 - accuracy: 0.4136 - val_loss: 1.4787 - val_accuracy: 0.4641
Epoch 5/70
391/391 [==============================] - 80s 204ms/step - loss: 1.5701 - accuracy: 0.4339 - val_loss: 1.4249 - val_accuracy: 0.4814
Epoch 6/70
391/391 [==============================] - 79s 202ms/step - loss: 1.5237 - accuracy: 0.4518 - val_loss: 1.4062 - val_accuracy: 0.4957
Epoch 7/70
391/391 [==============================] - 78s 199ms/step - loss: 1.4914 - accuracy: 0.4611 - val_loss: 1.3620 - val_a